<a href="https://colab.research.google.com/github/NancyFulda/towards-neural-programming-interfaces/blob/master/notebooks/politics/politics_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Notebook training NPI on political views

This notebook is also an example on how to use the NPI project to train your own NPI in a way that can affect GPT2's sentiment or emotion. This notebook attempts to train the GPT2 model with NPI to possess a specific political view.

## Setup

### Prerequisite setup:

*   Ensure you selected a runtime with a GPU. You can do that by clicking Runtime -> Change runtime type ->  Hardware Accelerator -> GPU
*   After running the setup code in this section, restart the runtime by clicking Runtime -> Restart runtime.

After you have done the prerequisite setup, you will not need to rerun this section again.


In [ ]:
!git clone https://github.com/NancyFulda/towards-neural-programming-interfaces.git
!cd towards-neural-programming-interfaces && ./install_dependencies.sh

In [ ]:
# Download the NewB news source dataset
!git clone https://github.com/JerryWei03/NewB.git

### Post setup

***Note: You will need to restart the runtime on Colab to continue***

Click on Runtime -> Restart Runtime to do so.


In [1]:
# For dev purposes, enable autoreload of modules
%load_ext autoreload

In [14]:
import pandas as pd
import torch

%autoreload

# Constructing training dataset
from npi.dataset import NPIDatasetConstructor, NPIDatasetLoader

# Models
from npi.models import NPITrainingModels

# Training NPI
from npi.training import NPIStyleTrainer, NPITrainer

# Configuration
from npi.config import NPIConfig


Note that each step in training an NPI is conditioned on the variable toggles, so you can disable certain steps to not redo them again when running the whole notebook.

In [3]:
# Toggle steps
construct = True
train_class = True
test_class = True
train_npi = True


Below is where an NPIConfig object is defined. It holds configuration for the NPI that you can use for the different NPI training steps. Notice that it is injected intothe NPITrainingModels class. That class will load the necessary models using the configuration defined in NPIConfig.

In [5]:
model_layers = [5, 11]

device = torch.device(f"cuda:0")
config = NPIConfig(
    device, gpt_model="gpt2", perturbation_indices=model_layers, npi_name="politics"
)


In [9]:
# Initialize models for training and testing
models = NPITrainingModels(
    config,
)

## Create Dataset

### Get Data from NewB GitHub repo

Note that this step will be different depending on the data you would want to train the NPI on.

#### About Construct Data

The key here is to give `construct_data.construct_dataset` a tuple of `(input_text, label)`.

Currently, `construct_data.construct_dataset` will generate up to the `NPIConfig.num_iters` times of text from GPT based on the input text, and labels those activations with the label you assigned to the input text. This is assuming the text outputted by GPT2 was relevant to our input text.

#### Limitations
* The labels that are currently supported is just `1` and `0`. *You need to label the type of output you want the GPT2 with NPI model to create with `0`.*
* Currently, you will need to manually stop the dataset generation if you want to end it before it goes through your input
* If you don't have a dataset with labels, you will need to create that manually.
* Despite your configurations for which GPT layers to modify by the NPI, this step will generate the data from all GPT layers

In [ ]:
if construct:
    with open("./NewB/train_orig.txt", "r", newline="") as f:
        data = pd.read_csv(f, delimiter="\t", names=["view", "text"])
        data["view"] = data["view"].replace(
            to_replace={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}
        )
        data = data[data.view != 5]
        data = data.sample(frac=1)
        data_iter = zip(data["text"], data["view"])
        construct_data = NPIDatasetConstructor(config)
        construct_data.construct_dataset(data_iter, 4000) # Only creating 4000 training data points 
        # TODO: setting the limit of training data points is a WIP. You can manually stop the cell to stop the dataset generation.


## Train classifier from Dataset



In [ ]:
model_loader = NPITrainingModels(config)
dataset_loader = NPIDatasetLoader(config, target="style")
style_trainer = NPIStyleTrainer(config, class_lr=1e-5)
classifier = None
if train_class:
    classifier = style_trainer.train_classifier(
        model_loader,
        dataset_loader,
        num_epochs=5,
        #continue_epoch=4,  # Set epoch of classifier to load to continue training.
    )


### Test Classifier


In [8]:
_, test_loader, _, _ = dataset_loader.load_train_and_test_dataloaders()

classifier = model_loader.load_style_classifier() if not classifier else classifier
# When output=True, it will log the following in models/npi_models/style_classifier_output.log:
#   (generated text from the dataset)
#   truth={target_label} actual={model_output}
style_trainer.test_model(test_loader, classifier, torch.nn.BCELoss(), output=True)


(74786075.66666667, 0.48333333333333334)

## Training NPI

In [ ]:
num_epochs = 6
split_ratio = 0.25
batch_size = 5
headstart = 0  # set at 0 for debugging

if train_npi:
    trainer = NPITrainer(config, batch_size=batch_size, headstart=headstart)
    dataset_loader = NPIDatasetLoader(config, split_ratio=split_ratio)
    trainer.train_adversarial_npi(models, num_epochs, dataset_loader)


## Test NPI

Using the model loader, you can directly generate NPI text.


In [17]:
text = "Trump today was found"
print(f"\nOriginal GPT2 output: '{text}'\n{models.gpt2_generate_text(text)}")


Model: GPT 2 Vanilla: 100%|██████████| 100/100 [00:02<00:00, 36.98it/s]



Original GPT2 output: 'Trump today was found'
 to have been a liar.

The FBI has been investigating the Trump campaign's ties to Russia.

The Trump campaign has also been accused of using the word "radical" in a tweet.

"I am not a fan of the idea of a "big-screen" movie.

"I'm not going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say that I'm going to say


In [18]:
print(f"\nNPI GPT2 output: '{text}'\n{models.npi_generate_text(text)}")

Model: GPT2 with NPI: 100it [00:05, 18.87it/s]


NPI GPT2 output: 'Trump today was found'
 to have been a liar.

The FBI has been investigating the Trump campaign's ties to Russia.

The Trump campaign has also been accused of using the word "radical" in a tweet.

"I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am not a racist, I am
